In [ ]:
import pandas as pd
import pathlib
import os

In [2]:
path = r"E:\Downloads\OSM 事务"

In [15]:
import pandas as pd
import pathlib
import os
import chardet

# --- Setup: Define your directory path ---
# NOTE: Replace 'E:/Downloads/OSM 事务' with the actual path.
path = r'E:\Downloads\OSM 事务' 
# --- End of Setup ---

all_dfs = []
print(f"Reading CSV files from: {path}")

# Define the maximum number of bytes to read for encoding detection
CHUNK_SIZE_FOR_DETECTION = 100000 
COMMON_CHINESE_ENCODING = 'gbk' # A robust standard for Chinese characters

for file in pathlib.Path(path).glob("*.csv"):
    file_name = file.name
    
    # 1. Detect Encoding using chardet
    detected_encoding = 'utf-8' # Default fallback
    
    try:
        # Read a chunk of the file in binary mode ('rb')
        with open(file, 'rb') as f:
            raw_data = f.read(CHUNK_SIZE_FOR_DETECTION)
            
        # Analyze the raw bytes
        result = chardet.detect(raw_data)
        
        # Extract the detected encoding. Use a high confidence threshold.
        if result['confidence'] > 0.7:
             # Standardize detected encoding names
            detected_encoding = result['encoding'].lower()
        
        print(f"-> Detected encoding for {file_name}: {detected_encoding} (Confidence: {result['confidence']:.2f})")

    except Exception as e:
        print(f"-> ⚠️ Warning: Could not detect encoding for {file_name}. Using fallback 'utf-8'. Error: {e}")
        detected_encoding = 'utf-8' # Fallback to utf-8 if detection fails


    # 2. Read the CSV using the detected/fallback encoding
    try:
        df = pd.read_csv(file, encoding=detected_encoding)
        print(f"-> ✅ Successfully read: {file_name} using {detected_encoding}")
        all_dfs.append(df)

    except UnicodeDecodeError:
        # Fallback 1: Try the robust Chinese encoding 'gb18030'
        try:
            df = pd.read_csv(file, encoding=COMMON_CHINESE_ENCODING)
            all_dfs.append(df)
            print(f"-> ✅ Successfully read (used '{COMMON_CHINESE_ENCODING}' fallback): {file_name}")

        except UnicodeDecodeError:
            # Fallback 2: Try 'latin1' for non-Unicode/European files that might have been misclassified
            try:
                df = pd.read_csv(file, encoding='latin1')
                all_dfs.append(df)
                print(f"-> ✅ Successfully read (used 'latin1' fallback): {file_name}")
            except Exception as e:
                # Final failure handling for reading the file
                print(f"-> ❌ FINAL FAILURE: Could not read {file_name} after all attempts. Skipping file. Error: {e}")
                continue # Skip to the next file
    
    except Exception as e:
        # Handle other pd.read_csv errors (e.g., incorrect separator)
        print(f"-> ❌ Failed to read {file_name} due to a structural error. Skipping file. Error: {e}")
        continue


# --- 3. Concatenation and Saving ---

if all_dfs:
    print("\nConcatenating all successfully read DataFrames...")
    big_df = pd.concat(all_dfs, ignore_index=True)

    output_file_name = "combined.csv"
    output_path = os.path.join(path, output_file_name)
    
    # Save the combined DataFrame, ensuring the output uses a universal encoding (UTF-8)
    big_df.to_csv(output_path, index=False, encoding='utf-8')
    
    print(f"\n✅ Successfully created and saved the combined file:")
    print(f"   Output file: {output_path}")
    print(f"   Total rows in combined file: {len(big_df)}")
    print(f"   Total files successfully processed: {len(all_dfs)}")
else:
    print("\n⚠️ No CSV files were successfully read and concatenated.")

Reading CSV files from: E:\Downloads\OSM 事务
-> Detected encoding for CPJ_DE_TRANS_2025101002.csv: ascii (Confidence: 1.00)
-> ✅ Successfully read: CPJ_DE_TRANS_2025101002.csv using ascii
-> Detected encoding for CPJ_DE_TRANS_2025101401.csv: ascii (Confidence: 1.00)
-> ✅ Successfully read: CPJ_DE_TRANS_2025101401.csv using ascii
-> Detected encoding for CPJ_DE_TRANS_2025101501.csv: gb2312 (Confidence: 0.99)
-> ✅ Successfully read: CPJ_DE_TRANS_2025101501.csv using gb2312
-> Detected encoding for CPJ_DE_TRANS_2025101601.csv: gb2312 (Confidence: 0.99)
-> ✅ Successfully read: CPJ_DE_TRANS_2025101601.csv using gb2312
-> Detected encoding for CPJ_DE_TRANS_2025102101.csv: gb2312 (Confidence: 0.99)
-> ✅ Successfully read: CPJ_DE_TRANS_2025102101.csv using gb2312
-> Detected encoding for CPJ_DE_TRANS_2025102102.csv: gb2312 (Confidence: 0.99)
-> ✅ Successfully read: CPJ_DE_TRANS_2025102102.csv using gb2312
-> Detected encoding for CPJ_DE_TRANS_2025102201.csv: ascii (Confidence: 1.00)
-> ✅ Succes